In [41]:
class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size
        self.children = []
        self.isDirectory = False if '.' in name else True
        self.extension = name.split('.')[1] if '.' in name else ''
    
    def __repr__(self):
        return self.name
    
#     def __len__(self):
#         return self.size

class Filter:
#     def __init__(self):
#         pass
    
    def apply(self, file):
        if not isinstance(file, File):
            raise TypeError

class MinSizeFilter(Filter):
    def __init__(self, size):
        self.size = size
    
    def apply(self, file):
        super().apply(file)
        return file.size > self.size

class ExtensionFilter(Filter):
    def __init__(self, extension):
        self.extension = extension
    
    def apply(self, file):
        super().apply(file)
        return file.extension == self.extension
    
class FileSystem:
    def __init__(self):
        self.filters = []
    
    def addFilter(self, givenFilter):
        if isinstance(givenFilter, Filter):
            self.filters.append(givenFilter)
        else:
            raise TypeError
    
    def applyOrFiltering(self, root):
        ret = []
        def dfs(node):
            if node.isDirectory:
                for child in node.children:
                    dfs(child)
            else:
                for givenFilter in self.filters:
                    if givenFilter.apply(node):
                        ret.append(node)
                        return
        dfs(root)
        return ret
    
    def applyAndFiltering(self, root):
        ret = []
        def dfs(node):
            if node.isDirectory:
                for child in node.children:
                    dfs(child)
            else:
                for givenFilter in self.filters:
                    if not givenFilter.apply(node):
                        return
                ret.append(node)
        dfs(root)
        return ret

In [42]:
f1 = File("root_300", 300)

f2 = File("fiction_100", 100)
f3 = File("action_100", 100)
f4 = File("comedy_100", 100)
f1.children=[f2,f3,f4]

f5 = File("StarTrek_4.txt", 4)
f6 = File("StarWars_10.xml", 10)
f7 = File("JusticeLeague_15.txt", 15)
f8 = File("Spock_1.jpg", 1)
f2.children=[f5,f6,f7,f8]

f9 = File("IronMan_9.txt", 9)
f10 = File("MissionImpossible_10.rar", 10)
f11 = File("TheLordOfRings_3.zip", 3)
f3.children=[f9,f10,f11]

f11 = File("BigBangTheory_4.txt", 4)
f12 = File("AmericanPie_6.mp3", 6)
f4.children=[f11,f12]


greater5 = MinSizeFilter(5)
txtFilter = ExtensionFilter("txt")

myFileSystem = FileSystem()
myFileSystem.addFilter(greater5)
myFileSystem.addFilter(txtFilter)

In [43]:
print(myFileSystem.applyOrFiltering(f1))
print(myFileSystem.applyAndFiltering(f1))

[StarTrek_4.txt, StarWars_10.xml, JusticeLeague_15.txt, IronMan_9.txt, MissionImpossible_10.rar, BigBangTheory_4.txt, AmericanPie_6.mp3]
[JusticeLeague_15.txt, IronMan_9.txt]


In [1]:
import bisect

class Locker:
    def __init__(self, x, y, z, id=None):
        self.length = x
        self.width = y
        self.height = z
        self.locker_id = id
        self.vacant = True
    
    def get_max_dimension(self):
        return max([self.length, self.width, self.height])
    
class Package:
    def __init__(self, x, y, z, id=None):
        self.length = x
        self.width = y
        self.height = z
        self.package_id = id
        
class LockerLocator:
    def __init__(self, lockers):
        self.lockers = lockers
        self.sorted_lockers = sorted((locker.get_max_dimesion, locker) for locker in self.lockers)
    
    def find_locker_for_box(self, box):
        maxd = max([box.length, box.width, box.height])
        index = bisect.bisect_left(self.sorted_lockers, maxd)
        for i in range(index, len(self.sorted_lockers)):
            if not self.sorted_lockers[i][1].vacant:
                continue
            if self.check_fit(self.sorted_lockers[i][1], box):
                self.sorted_lockers[i][1].vacant = False
                return self.sorted_lockers[i][1].id
    
    def check_fit(self, locker, box):
        locker_dimensions = sorted([locker.length, locker.width, locker.heigth])
        box_dimensions = sorted([box.length, box.width, box.height])
        for i in range(3):
            if box_dimensions[i] > locker_dimensions[i]:
                return False
        return True